# **Sentimantal Analysis on IMDB Dataset Movie Review using Recurrent Neural Network (RNN)**


#**Importing Necessary libraries**


In [7]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN, Dropout # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import re

# **Preparing the data named IMDB**

In [8]:
data = pd.read_csv('/content/IMDB Dataset.csv')
print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


Stop Word is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)
Declaring the english stop words

In [9]:
import nltk
nltk.download("stopwords")
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Load and Clean Dataset**
**In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in load_dataset() function, beside loading the dataset using pandas, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.**

# **Encode Sentiments**
**In the same function, We also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.**

In [11]:
def load_dataset():
    df = pd.read_csv('/content/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


/tmp/ipython-input-251577464.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


#**Split Dataset**
**In this work, We decided to split the data into 80% of Training and 20% of Testing set using train_test_split method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.**

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
40595    [crazy, director, yeah, need, crazy, make, nea...
30257    [to, anyone, might, think, show, please, give,...
36993    [a, cowboy, sympathetic, plight, nearby, india...
48093    [otto, preminger, directs, light, feather, sto...
2650     [i, thought, comedy, what, hoot, i, believe, f...
                               ...                        
11535    [a, kind, road, movie, old, fashioned, trains,...
18664    [this, picture, seemed, way, slanted, almost, ...
35163    [firstly, although, many, say, worst, series, ...
24978    [having, spent, six, years, previous, writing,...
2230     [when, thinking, revelation, main, character, ...
Name: review, Length: 40000, dtype: object 

22338    [a, long, time, ago, galaxy, far, far, away, t...
42641    [father, hood, entertaining, tale, unwilling, ...
38986    [this, far, worst, piece, cr, p, i, ever, seen...
36861    [uh, oh, another, gay, film, this, time, showi...
33679    [this, basically, attempt, thing, batman, done...
 

**Function for getting the average review length, by calculating the mean of all the reviews length (using numpy.mean)**

In [13]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

#**Tokenize and Pad/Truncate Reviews**
**A Neural Network only accepts numeric data, so we need to encode the reviews. I use tensorflow.keras.preprocessing.text.Tokenizer to encode the reviews into integers, where each unique word is automatically indexed (using fit_on_texts method) based on x_train.**

**x_train and x_test is converted into integers using texts_to_sequences method.**

**Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using tensorflow.keras.preprocessing.sequence.pad_sequences.**

In [14]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding
print('Total Words:', total_words)

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Total Words: 92375
Encoded X Train
 [[ 821   69 1083 ...    0    0    0]
 [ 282  152  138 ...    0    0    0]
 [  39 2674 2204 ...    0    0    0]
 ...
 [4808  163   37 ...    0    0    0]
 [1533  909 1324 ... 1131 2482  290]
 [ 170  435 3164 ...    0    0    0]] 

Encoded X Test
 [[   39   103    10 ...    49  3571   253]
 [  236  2663   341 ... 16316 11068   204]
 [    8   132   153 ...     0     0     0]
 ...
 [    1   310   130 ...     0     0     0]
 [  699   102    83 ...     0     0     0]
 [  314  2145   307 ...     0     0     0]] 

Maximum review length:  130


#**Build Architecture/Model**
**Embedding Layer: in simple terms, it creates word vectors of each word in the word_index and group words that are related or have similar meaning by analyzing other words around them.**

**RNN Layer: to make a decision to keep or throw away data by considering the current input, previous output.**

**Dense Layer: compute the input with the weight matrix and bias (optional), and using an activation function. I use Sigmoid activation function for this work because the output is only 0 or 1.**

**The optimizer is Adam and the loss function is Binary Crossentropy because again the output is only 0 and 1, which is a binary number.**

In [15]:
# -----------------------------
# Model Definition
# -----------------------------
model = Sequential([
    Embedding(total_words, 32, input_length=max_length),

    SimpleRNN(64, return_sequences=True, activation="tanh"),
    Dropout(0.5),

    SimpleRNN(32, return_sequences=True, activation="tanh"),
    Dropout(0.5),

    SimpleRNN(16, activation="tanh"),
    Dropout(0.4),

    Dense(1, activation="sigmoid")
])

model.summary()

# -----------------------------
# Compile Model
# -----------------------------
model.compile(
    optimizer=Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

#**Trainin the Model**

In [16]:
import sys
sys.stderr = open('/dev/null', 'w')

In [17]:
# -----------------------------
# Callbacks
# -----------------------------

early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    filepath="best_rnn_model.keras",  # modern Keras format
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# -----------------------------
# Training
# -----------------------------
history = model.fit(
    x_train,
    y_train,
    epochs=25,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

Epoch 1/25
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4980 - loss: 0.7982
Epoch 1: val_accuracy improved from -inf to 0.50087, saving model to best_rnn_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.4981 - loss: 0.7979 - val_accuracy: 0.5009 - val_loss: 0.6951
Epoch 2/25
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5082 - loss: 0.7142
Epoch 2: val_accuracy did not improve from 0.50087
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.5081 - loss: 0.7142 - val_accuracy: 0.4902 - val_loss: 0.6952
Epoch 3/25
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5251 - loss: 0.6958
Epoch 3: val_accuracy improved from 0.50087 to 0.50575, saving model to best_rnn_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.5251 - loss: 0.6958 - val_accuracy: 0.5058 - val_loss: 0.6964
Epoch 4/25
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5897 - loss: 0.6652
Epoch 4: val_accuracy did not improve from 0.50575
250/250 ━━━

#**Saving The Model**

In [18]:
model = model.save('rnn.h5')
loaded_model = load_model('rnn.h5')

#**Evaluation**

In [19]:
# 1. Predict probabilities
y_pred_prob = loaded_model.predict(x_test, batch_size=128)

# 2. Convert probabilities to class labels (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# 3. Convert y_test to numpy array (if it's a pandas Series)
y_true = y_test.values if hasattr(y_test, "values") else y_test

# 4. Sanity check (optional but recommended)
print("Predictions (first 10):", y_pred[:10])
print("Ground Truth (first 10):", y_true[:10])

# 5. Calculate accuracy
correct = np.sum(y_pred == y_true)
wrong = len(y_pred) - correct
accuracy = (correct / len(y_pred)) * 100

# 6. Print results
print("Correct Prediction:", correct)
print("Wrong Prediction:", wrong)
print(f"Accuracy: {accuracy:.2f}%")


79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step
Predictions (first 10): [1 1 1 0 1 1 0 0 0 1]
Ground Truth (first 10): [1 1 0 0 1 0 1 0 0 0]
Correct Prediction: 5094
Wrong Prediction: 4906
Accuracy: 50.94%


#**Example review**

Message: **Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!**

**A complete waste of time. Nothing made sense and the film felt painfully slow and pointless.**



In [21]:
review = str(input('Movie Review: '))

Movie Review: 


#**Pre-processing of entered review**

In [22]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  
Filtered:  ['']


In [23]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


#**Prediction**

In [24]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
[[0.5040976]]


In [25]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative
